In [1]:
import essentia
print(essentia.__version__)
print(essentia.__file__)
import essentia.standard as es

# let's have a look at what is in there
#print(dir(essentia.standard))

import json
import matplotlib.pyplot as plt
import numpy as np
import cv2

import utils.util as util
import moviepy.editor as mp

2.1-beta6-dev
/usr/local/lib/python3.8/site-packages/essentia/__init__.py


2023-04-04 01:04:54.715006: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
''' HELPERS '''
#print(dir(essentia.standard))
#print(help(es.MonoLoader))

In [2]:
def predicition_complete(audio,nlabels2predict,printt=False,plott=False,save_plot=False):
    
    # Loading the model
    graph_filename = "/raid/DATASETS/.zuble/vigia/zuwav/fsd-sinet-essentia/models/fsd-sinet-vgg41-tlpf-1.pb"
    model = es.TensorflowPredictFSDSINet(graphFilename=graph_filename)

    # Read the metadata
    metadata_file = "/raid/DATASETS/.zuble/vigia/zuwav/fsd-sinet-essentia/models/fsd-sinet-vgg41-tlpf-1.json"
    metadata = json.load(open(metadata_file, "r"))
    labels = metadata["classes"]
    
    
    def top_from_average(data, top_n):
        av = np.mean(data, axis=1)
        sorting = np.argsort(av)[::-1]
        return sorting[:top_n], [av[i] for i in sorting]

    
    def plot_predictions(top_preds, top_labels_w_av):
    
        # Generate plots and improve formatting
        matfig = plt.figure(figsize=(8, 3))
        plt.matshow(top_preds, fignum=matfig.number, aspect="auto")

        plt.yticks(np.arange(len(top_labels_w_av)), top_labels_with_av)
        locs, _ = plt.xticks()
        ticks = np.array(locs // 2).astype("int")
        plt.xticks(locs[1: -1], ticks[1: -1])
        plt.tick_params(
            bottom=True, top=False, labelbottom=True, labeltop=False
        )
        plt.xlabel("(s)")

        if save_plot : plt.savefig("activations.png", bbox_inches='tight')
    
    
    predictions = model(audio)
    return predictions
    if printt:
        for label, probability in zip(metadata['classes'], predictions.mean(axis=0)):
            print(f'{label}: {100 * probability:.1f}%') 

    # Compute the top-n labels and predictions
    top_n, averages = top_from_average(predictions,nlabels2predict)
    
    top_labels = [labels[i] for i in top_n]
    if printt : print(top_labels)
    
    top_labels_with_av = [
        f"{label} ({av:.3f})" for label, av in zip(top_labels, averages)
    ]
    if printt: print(top_labels_with_av)
    
    top_predictions = np.array([predictions[i, :] for i in top_n])
    if plott: plot_predictions(top_predictions, top_labels_with_av)
    
    return top_labels_with_av




In [3]:
test_mp4_paths,test_mp4_labels,test_aac_paths,test_aac_labels = util.load_xdv_test(util.SERVER_TEST_AUD_ORIG_PATH)

# Extract the audio from the video
audio = mp.AudioFileClip(filename=test_mp4_paths[10])#.fx(mp.afx.audio_normalize)
audio_total_array = audio.to_soundarray()
audio_total_array_mono = np.mean(audio_total_array, axis=1)
audio_total_array_mono_single = audio_total_array_mono.astype(np.float32)

predictions = predicition_complete(audio_total_array_mono_single,2)


acc_path /raid/DATASETS/anomaly/XD_Violence/aud/testing/original


[   INFO   ] TensorflowPredict: Successfully loaded graph file: `/raid/DATASETS/.zuble/vigia/zuwav/fsd-sinet-essentia/models/fsd-sinet-vgg41-tlpf-1.pb`
2023-04-04 01:05:03.203500: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2198820000 Hz


In [ ]:
tt = ['/raid/DATASETS/anomaly/XD_Violence/testing_copy/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A_1.mp4']
util.print_acodec_from_mp4(tt,printt=True)